In [1]:
from pruning import *
from lw_retrain_utils import *
from evaluation import *
import json
import copy 
from datasets import load_dataset
import os
import gc
import shutil

/users/eleves-a/2021/ahmad.rammal/miniconda3/envs/colab/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "openai-community/gpt2-medium"
teacher_model, tokenizer = load_model(model_name)

In [ ]:
student_models = ["depth=16", "depth=20_num_heads=12_mlp_exp=3.5_embed_size=1024", "num_heads=12_mlp_exp=2.5_embed_size=1024"]
params =[
    {"depth": 16},
    {"depth": 20, "num_heads": 12, "mlp_exp": 3.5, "embed_size": 1024},
    {"num_heads": 12, "mlp_exp": 2.5, "embed_size": 1024}
]

In [ ]:
student_model_name = f"saved_models/{student_models[0]}/checkpoint-280/model.safetensors"
student_model, tokenizer = load_model(model_name)

In [13]:
for i in range(student_model.config.n_layer-params[0]["depth"]):
    student_model.transformer.h.pop(0)

In [20]:
from safetensors.torch import safe_open

try:
    with safe_open(student_model_name, framework="pt") as f:
        print("Checkpoint keys:", list(f.keys()))
except Exception as e:
    print("Safe opening failed:", e)


Safe opening failed: Error while deserializing header: HeaderTooLarge


In [21]:
from safetensors.torch import load_file


try:
    state_dict = load_file(student_model_name)
    print("Checkpoint keys:", state_dict.keys())  # List available tensors
except Exception as e:
    print("Error loading checkpoint:", e)


Error loading checkpoint: Error while deserializing header: HeaderTooLarge
